# Identificação de Estilos Textuais
O objetivo do projeto é construir um classificador que seja capaz de identificar o estilo o qual o texto pertença. Serão criados três classificadores, para estes estilos textuais:
- Arcaico x Moderno
- Simples x Complexo
- Literal x Dinâmico

In [1]:
#Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Arcaico X Moderno
Primeiramente, será investigado a resolução do problema de identificar textos Arcaicos e Modernos.

In [2]:
file_path = "treino/train_arcaico_moderno.csv"
df = pd.read_csv(file_path)

In [3]:
df.head()

,text;style
0,És a fonte dos jardins; poço das águas vivas; ...
1,Anda com os sábios e serás sábio; mas o compan...
2,Porque eu; o Senhor; não mudo; por isso; vós; ...
3,E; ainda que nunca viu o sol; nem o conheceu; ...
4,todo o reino de Ogue; em Basã; que reinou em A...


Para manipular corretamente os dados textuais, será necessário construir um dataframe que possui o texto bíblico separado do rótulo

In [4]:
df[['text', 'target']] = df['text;style'].str.rsplit(";", n=1, expand=True) #divide apenas 1 vez, começando pela direita

In [5]:
df.drop('text;style', axis=1, inplace=True)

In [6]:
df['target'].value_counts()

target
arcaico    18442
moderno    18442
Name: count, dtype: int64

Portanto, percebe-se que o conjunto de textos bíblicos é balanceado. Logo, a métrica de acurácia é confiável.

Primeiramente, iremos modelar a representação textual que será utilizado para prever o estilo do texto bíblico

In [7]:
#Modelo Bag-of-Words
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
X = vect.fit_transform(df['text']) #Gera representação Bag-of-Words com frequência normalizada

In [8]:
X.shape

(36884, 27703)

Foi gerado um representação Bag-of-Words com um vocabulário de 27.703 palavras (tokens). Será utilizado um classificador simples como a Regressão Logística, somente para avaliar quão boa é a representação escolhida.

In [9]:
Y = df['target'] #estilo "arcaico x moderno"

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_number = le.fit_transform(Y)

In [11]:
le.classes_ # Classe positiva é Moderno

array(['arcaico', 'moderno'], dtype=object)

In [12]:
type_metrics = [
    'accuracy',
    'precision',
    'recall',
    'f1', 
    'f1_macro'
]

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
results = cross_validate(
    estimator=model,
    X = X,
    y = y_number,
    cv=5,
    scoring=type_metrics,
    return_train_score=False
)

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.or

In [14]:
results_df = pd.DataFrame(results)
results_df.drop(['fit_time', 'score_time'], axis=1, inplace=True)
results_df

,test_accuracy,test_precision,test_recall,test_f1,test_f1_macro
0,0.849397,0.850041,0.848427,0.849233,0.849397
1,0.851430,0.848387,0.855748,0.852052,0.851427
2,0.849668,0.847334,0.853077,0.850196,0.849666
3,0.848855,0.852024,0.844402,0.848196,0.848852
4,0.840835,0.837178,0.846258,0.841694,0.840830


Como é possível perceber, com o uso de BoW sem TF-IDF, obteve-se um bom desempenho, inclusive não priorizando nenhuma classe. Agora, iremos testar BoW TF-IDF.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer()
X_tf = tf_idf.fit_transform(df['text']) #Representação Bag-of-Word com uso de TF-IDF

In [16]:
X_tf.shape #Exata quantidade de tokens comparado ao BOW Frequency

(36884, 27703)

In [17]:
results = cross_validate(
    estimator=model,
    X= X_tf,
    y= y_number,
    cv=5,
    scoring=type_metrics,
    return_train_score=False
)

In [18]:
r_tfidf = pd.DataFrame(results)
r_tfidf.drop(['fit_time', 'score_time'], axis=1, inplace=True)
r_tfidf

,test_accuracy,test_precision,test_recall,test_f1,test_f1_macro
0,0.840043,0.837823,0.843275,0.840541,0.840042
1,0.841941,0.838978,0.846258,0.842603,0.841938
2,0.841399,0.834884,0.851179,0.842953,0.841383
3,0.836654,0.833512,0.841420,0.837448,0.836651
4,0.838802,0.830119,0.851952,0.840894,0.838774


Como é possível visualizar, o BoW e o TF-IDF obtiveram desempenhos próximos. Agora, iremos partir para explorar o uso de n_gramas. Iremos testar os seguintes intervalos de n_gramas:
- (1, 2): unigramas, bigramas
- (1, 3): unigramas, bigramas e trigramas

In [19]:
tf_gr1 = TfidfVectorizer(ngram_range=(1,2))
X_gr1 = tf_gr1.fit_transform(df['text']) # Uso de unigramas e bigramas no TF-IDF
X_gr1.shape

(36884, 295411)

In [20]:
results = cross_validate(
    estimator=model,
    X= X_gr1,
    y= y_number,
    cv=5,
    scoring=type_metrics,
    return_train_score=False
)

In [21]:
r_gr1 = pd.DataFrame(results)
r_gr1.drop(['fit_time', 'score_time'], axis=1, inplace=True)
r_gr1

,test_accuracy,test_precision,test_recall,test_f1,test_f1_macro
0,0.846821,0.839979,0.856833,0.848322,0.846806
1,0.850346,0.845085,0.857918,0.851453,0.850337
2,0.850481,0.842607,0.862022,0.852204,0.850461
3,0.846008,0.835303,0.862022,0.848453,0.845968
4,0.849919,0.838270,0.867137,0.852459,0.849874


In [22]:
tf_gr2 = TfidfVectorizer(ngram_range=(1,3))
x_gr2 = tf_gr2.fit_transform(df['text']) #Uso de unigramas, bigramas e trigramas no TF-IDF
x_gr2.shape

(36884, 795287)

In [23]:
results = cross_validate(
    estimator=model,
    X= x_gr2,
    y= y_number,
    cv=5,
    scoring=type_metrics,
    return_train_score=False
)

In [24]:
r_gr2 = pd.DataFrame(results)
r_gr2.drop(['fit_time', 'score_time'], axis=1, inplace=True)
r_gr2

,test_accuracy,test_precision,test_recall,test_f1,test_f1_macro
0,0.839094,0.827785,0.856291,0.841797,0.839048
1,0.840992,0.835960,0.848427,0.842148,0.840984
2,0.846143,0.836939,0.859854,0.848242,0.846114
3,0.839637,0.827667,0.857956,0.842540,0.839582
4,0.843953,0.829395,0.866052,0.847327,0.843877


Como é possível perceber, não houve diferença ao usar n-gramas. Pelo contrário, somente aumentar a dimensionalidade dos dados, o que causa o fenômeno chamado maldição da dimensionalidade. Agora, iremos investigar se escolher as características mais signitificativas melhoram o desempenho.

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV, StratifiedKFold

pipeline = Pipeline([
    ("select", SelectKBest(score_func=chi2)),
    ("clf", LogisticRegression(random_state=42))
])

In [26]:
param_grid = {
    "select__k": [10000, 15000, 17000, 20000, 22000, 25000]
}

In [27]:
grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="accuracy",
    n_jobs=-1
)

In [28]:
grid.fit(X_tf, y_number)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('select',
                                        SelectKBest(score_func=<function chi2 at 0x000001F202F9EA20>)),
                                       ('clf',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid={'select__k': [10000, 15000, 17000, 20000, 22000,
                                       25000]},
             scoring='accuracy')

In [31]:
grid.best_params_

{'select__k': 22000}

In [32]:
grid.best_score_

0.8270794647548796

Portanto, é possível ter desempenho semelhante ao original com 22.000 features. Logo, para o desenvolvimento do classificador, será utilizado um BoW TF-IDF com o uso de SelectKBest, com K = 22000